<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/GenAI_Practice/Langwatch/try_dspy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try DsPy for RAG Prompt Optimization


In [1]:
%%capture --no-stderr
!pip install --upgrade nbformat
%pip install -U --quiet dspy

## Prepare LLM

* `http://20.102.90.50:2017/wiki17_abstracts` provides the sources for retrieval here

In [2]:
import os
import pandas as pd
from getpass import getpass
import dspy
from google.colab import userdata


# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# llm = dspy.LM("openai/gpt-4.1-nano", api_key=OPENAI_API_KEY)

GOOGLE_AI_API_KEY = userdata.get('GOOGLE_AI_API_KEY')
llm = dspy.LM("gemini/gemini-2.0-flash", api_key=GOOGLE_AI_API_KEY)
print("LLM test response:", llm("Where's Silicon Valley?"))

# the retrieval model
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

LLM test response: ['Silicon Valley is located in the southern part of the San Francisco Bay Area in **Northern California, United States**.\n']


## Preparing Dataset

In [3]:
from dspy.datasets import HotPotQA


dataset = HotPotQA(train_seed=1, train_size=32, eval_seed=2025, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

print()
print(len(trainset), len(devset))
print(trainset[0])
print(devset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]


32 50
Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Example({'question': 'Pehchaan: The Face of Truth stars Vinod Khanna, Rati Agnihotri and which Indian actress, producer, and former model who also produced the film?', 'answer': 'Raveena Tandon', 'gold_titles': {'Raveena Tandon', 'Pehchaan: The Face of Truth'}}) (input_keys={'question'})


## Defining DsPy RAG

In [4]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context,
                               answer=prediction.answer,
                               reasoning=prediction.reasoning)


dev_example = devset[12]
print(f"[Devset] Question: {dev_example.question}")
print(f"[Devset] Answer: {dev_example.answer}")
print(f"[Devset] Relevant Wikipedia Titles: {dev_example.gold_titles}")
print()

generate_answer = RAG()
pred = generate_answer(question=dev_example.question)
print(f"[Prediction] Question: {dev_example.question}")
print(f"[Prediction] Predicted Answer: {pred.answer}")
print(f"[Prediction] Reasoning: {pred.reasoning}")

[Devset] Question: Twelve Inches is a compilation album by which 1980s British band?
[Devset] Answer: Frankie Goes to Hollywood
[Devset] Relevant Wikipedia Titles: {'Twelve Inches', 'Frankie Goes to Hollywood'}

[Prediction] Question: Twelve Inches is a compilation album by which 1980s British band?
[Prediction] Predicted Answer: Bananarama
[Prediction] Reasoning: The question asks which 1980s British band released a compilation album called "Twelve Inches". I need to find a band that matches both criteria.
The context provides three albums with "Twelve Inch" in the title:
- The Twelve Inch Singles by Soft Cell
- The Twelve Inches of Bananarama by Bananarama
- The Twelve Inch Mixes by Spandau Ballet

All three bands are British and were active in the 1980s. However, the question asks for the album title "Twelve Inches", so the answer must be Bananarama.


In [5]:
from dspy.teleprompt import MIPROv2


trial_logs = []

def validate_context_and_answer(example, prediction):
    gold = example.answer.strip().lower()
    pred = prediction.answer.strip().lower()
    score = int(gold == pred)

    # Format similar to Langwatch's internal logging
    log_entry = {
        "input": {
            "question": example.question,
            "context": getattr(example, "context", "")
        },
        "output": {
            "answer": pred
        },
        # Include prediction trace (as dict if possible)
        "trace": prediction.__dict__ if hasattr(prediction, "__dict__") else str(prediction),
        "score": score,
        "optimizer_name": optimizer.__class__.__name__,
    }
    trial_logs.append(log_entry)

    print(f"[Trial] Q: {example.question} | Pred: {pred} | GT: {gold} | Score: {score}")
    return score


optimizer = MIPROv2(
    metric=validate_context_and_answer,
    prompt_model=llm,
    task_model=llm,
    num_candidates=2,
    init_temperature=0.7,
    auto=None,
)


compiled_rag = optimizer.compile(
    RAG(),
    trainset=trainset,
    num_trials=5,
    max_bootstrapped_demos=2,
    max_labeled_demos=3,
    minibatch_size=4,
    requires_permission_to_run=False
)

2025/05/31 16:30:46 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/31 16:30:46 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/31 16:30:46 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=2 sets of demonstrations...
2025/05/31 16:30:46 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/05/31 16:30:46 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


Bootstrapping set 1/2
Bootstrapping set 2/2


2025/05/31 16:30:48 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing N=2 instructions...

2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: 0: Answer questions with short factoid answers.

2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: 1: Given relevant context, answer the question with a short, fact-based answer. Explain your reasoning step-by-step before providing the final answer.

2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: ==> STEP 3: FINDING OPTIMAL PROMPT PARAMETERS <==
2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: We will evaluate the program over a series of trials with different combinations of instructions and few-shot examples to find the optimal combination using Bayesian Optimization.

2025/05/31 16:31:10 INFO dspy.teleprompt.mipro_optimizer_v2: == Tri

Average Metric: 4.00 / 10 (40.0%):  40%|████      | 10/25 [00:05<00:08,  1.78it/s]

2025/05/31 16:31:18 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:19 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:19 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:19 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:20 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:20 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:20 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31:20 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:31

...

2025/05/31 16:32:14 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:32:15 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:32:16 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/31 16:32:18 ERROR dspy.utils.parallelizer: Error for Example({'question': 'Which band had a longer hiatus, Juliette and the Licks or The Last Shadow Puppets?', 'answer': 'The Last Shadow Puppets'}) (input_keys={'question'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini

Average Metric: 6.00 / 14 (42.9%):  92%|█████████▏| 23/25 [00:21<00:02,  1.34s/it]

2025/05/31 16:32:20 ERROR dspy.utils.parallelizer: Error for Example({'question': 'Which movie was released first, Son of Flubber or Davy Crockett, King of the Wild Frontier?', 'answer': 'Davy Crockett, King of the Wild Frontier'}) (input_keys={'question'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProje

Average Metric: 6.00 / 14 (42.9%):  96%|█████████▌| 24/25 [00:22<00:00,  1.06it/s]

2025/05/31 16:32:20 WARNING dspy.utils.parallelizer: Execution cancelled due to errors or interruption.
2025/05/31 16:32:20 ERROR dspy.teleprompt.utils: An exception occurred during evaluation
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/dspy/teleprompt/utils.py", line 52, in eval_candidate_program
    return evaluate(candidate_program, devset=trainset, return_all_scores=return_all_scores, callback_metadata={"metric_key": "eval_full"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/dspy/utils/callback.py", line 326, in sync_wrapper
    return fn(instance, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/dspy/evaluate/evaluate.py", line 171, in __call__
    results = executor.execute(process_item, devset)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [10]:
# optimized results
compiled_rag

generate_answer.predict = Predict(StringSignature(context, question -> reasoning, answer
    instructions='Answer questions with short factoid answers.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [7]:
# example output with optimized results
dev_example = devset[0]
pred = compiled_rag(question=dev_example.question)
print("\n--- Test on dev example ---")
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Ground Truth: {dev_example.answer}")


--- Test on dev example ---
Question: Pehchaan: The Face of Truth stars Vinod Khanna, Rati Agnihotri and which Indian actress, producer, and former model who also produced the film?
Predicted Answer: Raveena Tandon
Ground Truth: Raveena Tandon


In [11]:
from pprint import pprint

print(len(trial_logs))
pprint(trial_logs[:2])

40
[{'input': {'context': '',
            'question': 'Which American actress who made their film debut in '
                        'the 1995 teen drama "Kids" was the co-founder of Voto '
                        'Latino?'},
  'optimizer_name': 'MIPROv2',
  'output': {'answer': 'rosario dawson'},
  'score': 1,
  'trace': {'_completions': None,
            '_lm_usage': None,
            '_store': {'answer': 'Rosario Dawson',
                       'context': ['Rosario Dawson | Rosario Isabel Dawson '
                                   '(born May 9, 1979) is an American actress, '
                                   'producer, singer, comic book writer, and '
                                   'political activist. She made her film '
                                   'debut in the 1995 teen drama "Kids". Her '
                                   'subsequent film roles include "He Got '
                                   'Game", "Men in Black II", "25th Hour", '
                          

In [12]:
print(f"Total prompts sent: {len(llm.history)}")
llm.inspect_history(n=1)

Total prompts sent: 51




[2025-05-31T16:32:52.919519]

System message:

Your input fields are:
1. `context` (str): may contain relevant facts
2. `question` (str)
Your output fields are:
1. `reasoning` (str)
2. `answer` (str): often between 1 and 5 words
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer questions with short factoid answers.


User message:

[[ ## context ## ]]
[1] «Pehchaan: The Face of Truth | Pehchaan: The Face of Truth is a Bollywood film released in 2005. The film directed by Shrabani Deodhar stars Vinod Khanna, Rati Agnihotri and Raveena Tandon who also produced the film.»
[2] «Mashaal | Mashaal is a 1984 Bollywood film. Produced and directed by Yash Chopra, it starred Dilip Kumar, Anil Kapoor, Waheeda R

In [15]:
history_records = []

ct = 0
for record in llm.history:
  ct += 1

  pprint(record)
  print()
  if ct == 2:
    break

{'cost': 9.8e-06,
 'kwargs': {},
 'messages': None,
 'model': 'gemini/gemini-2.0-flash',
 'model_type': 'chat',
 'outputs': ['Silicon Valley is located in the southern part of the San '
             'Francisco Bay Area in **Northern California, United States**.\n'],
 'prompt': "Where's Silicon Valley?",
 'response': ModelResponse(id='chatcmpl-513e90af-3918-4f66-a274-49dbd63db162', created=1748708947, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Silicon Valley is located in the southern part of the San Francisco Bay Area in **Northern California, United States**.\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=23, prompt_tokens=6, total_tokens=29, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=6, image_tokens=None)), vertex_ai